# 📦 12_load_playlists_manual_static_to_bigquery

### 🎯 Objetivo

Este notebook no genera snapshots históricos.

Su propósito es cargar la tabla estructural playlists_manual_static a BigQuery para permitir joins analíticos con las tablas snapshot.

Esta tabla actúa como dimensión descriptiva de playlists manuales y se reescribe completamente en cada ejecución del pipeline mensual.  

Destino: youtube-datasets-360.angelgarciadatablog.playlists_manual_static

In [ ]:
from dotenv import load_dotenv
import os
from google.cloud import bigquery

In [ ]:
load_dotenv()

PROJECT_ID = os.getenv("GCP_PROJECT")
DATASET_ID = "angelgarciadatablog"
TABLE_ID = "playlists_manual_static"

FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)

print("Destino configurado:", FULL_TABLE_ID)


## 🧱 Cargar snapshot desde Parquet (temporal)  

⚠️ Nota temporal:
Durante la fase de notebooks, el DataFrame se carga desde Parquet como mecanismo de intercambio entre notebooks.
En la versión productiva (scripts .py), el DataFrame se pasará directamente sin almacenamiento intermedio.

In [ ]:
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd().parents[0]
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

df_playlists_manual_static = pd.read_parquet(
    PROCESSED_PATH / "playlists_manual_static.parquet"
)

df_playlists_manual_static.head()


In [ ]:
df_playlists_manual_static.dtypes

## 🏗 Crear tabla particionada con el esquema y datos del dataframe 

En este caso definimos el esquema manualmente en lugar de dejar que BigQuery lo infiera automáticamente desde el DataFrame. Esto nos permite controlar explícitamente los tipos de datos, especialmente published_at y extracted_at, que queremos almacenar como DATETIME (hora local Perú) y no como TIMESTAMP. Cuando BigQuery infiere el esquema, puede convertir columnas datetime en TIMESTAMP, lo que introduce interpretación en UTC. Definir el esquema manualmente garantiza consistencia y evita ambigüedades en el modelo temporal.

In [ ]:
from google.api_core.exceptions import NotFound
from google.cloud.bigquery import SchemaField

schema = [
    SchemaField("playlist_id", "STRING"),
    SchemaField("channel_id", "STRING"),
    SchemaField("title", "STRING"),
    SchemaField("description", "STRING"),
    SchemaField("item_count", "INT64"),
    SchemaField("privacy_status", "STRING"),
    SchemaField("published_at", "TIMESTAMP"),
    SchemaField("thumbnail_url", "STRING"),
    SchemaField("playlist_url", "STRING"),
    SchemaField("extracted_at", "TIMESTAMP"),
]

try:
    client.get_table(FULL_TABLE_ID)
    print("La tabla ya existe.")

except NotFound:
    table = bigquery.Table(FULL_TABLE_ID, schema=schema)
    client.create_table(table)
    print("Tabla creada correctamente.")



## 📌 Cargar datos del parquet a big query

In [ ]:
# 2️⃣ Carga los datos desde tu DataFrame hacia BigQuery. WRITE TRUNCATE = sobreescribe los datos
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

job = client.load_table_from_dataframe(
    df_playlists_manual_static,
    FULL_TABLE_ID,
    job_config=job_config
)

job.result()

print("Tabla playlists_manual_static reemplazada correctamente.")

